# Capture d'informations à partir des lettres majuscules avec l'embedding de caractères


Une autre amélioration potentielle consiste à appliquer l'embedding de caractères. Une représentation d'embeddings de mots pré-entraînée, telle que word2vec, peut nécessiter de transformer les mots en minuscules. Pourtant, comme le font les humains, les lettres majuscules contiennent des informations utiles pour déterminer si un mot est une entité nommée ou non. Par conséquent, dans ce notebook, nous incluons l'embedding au niveau des caractères avec Convolutional Neural Network (CNN). 

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

ModuleNotFoundError: No module named 'google.colab'

In [2]:
!pip install torchtext==0.6.0

import time
import torch
from torch import nn
from torch.optim import Adam
from torchtext.data import Field, NestedField, BucketIterator
from torchtext.datasets import SequenceTaggingDataset
from torchtext.vocab import Vocab
from collections import Counter

import torch
import torch.nn as nn
import torch.optim as optim

from torchtext import data
from torchtext import datasets

import spacy
import numpy as np
import pandas as pd

import time
import random
import string
from itertools import chain

     |████████████████████████████████| 1.1MB 1.7MB/s eta 0:00:01


## Préparer les données

Torchtext fournit une classe pratique pour gérer une entrée basée sur des caractères appelée `NestedField`. Pour inclure les caractères en tant que `NestedField`, nous devons créer une sorte de conteneur ` char_nesting_field`, puis spécifier cet objet comme paramètre pour le `NestedField`. 
Pour la tokenisation, on utilise la fonction intégrée `list` pour transformer le mot en caractères. La fonction de tokenisation peut être spécifiée dans le conteneur `Field`. 

On construit également le vocabulaire pour le `char_field`

In [3]:
# pour la reproductibilité
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [4]:
TEXT = data.Field(lower = True) 
TAG = data.Field(unk_token = None) # les tags sont tous connus on a alors unk_token = None
CHAR_NESTING= Field(tokenize=list)
CHAR = NestedField(CHAR_NESTING) 

In [5]:
train_data, valid_data, test_data = data.TabularDataset.splits(
        path="data_ner/",
        train="train.csv",
        validation="valid.csv",
        test="test.csv", format='csv', skip_header=True,
        fields=(
            (("text", "char"), (TEXT, CHAR)), 
            ("tag", TAG)
        )
    )

In [6]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")

Number of training examples: 14986
Number of validation examples: 3465
Number of testing examples: 3683


In [7]:
MIN_FREQ = 2

TEXT.build_vocab(train_data, 
                 min_freq = MIN_FREQ, # les mots qui apparaissent moins que MIN_FREQ fois seront ignorés du vocabulaire
                 vectors = "glove.6B.100d",
                 unk_init = torch.Tensor.normal_)


TAG.build_vocab(train_data)
CHAR.build_vocab(train_data) 

In [8]:
print(f"Unique tokens dans le vocabulaire de TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens dans le vocabulaire de TAG : {len(TAG.vocab)}")
print(f"Unique tokens dans le vocabulaire de CHAR : {len(CHAR.vocab)}")

Unique tokens dans le vocabulaire de TEXT vocabulary: 10952
Unique tokens dans le vocabulaire de TAG : 10
Unique tokens dans le vocabulaire de CHAR : 86


On affiche un exemple : 

In [9]:
print(vars(train_data.examples[1]))

{'text': ['eu', 'rejects', 'german', 'call', 'to', 'boycott', 'british', 'lamb', '.'], 'char': [['E', 'U'], ['r', 'e', 'j', 'e', 'c', 't', 's'], ['G', 'e', 'r', 'm', 'a', 'n'], ['c', 'a', 'l', 'l'], ['t', 'o'], ['b', 'o', 'y', 'c', 'o', 't', 't'], ['B', 'r', 'i', 't', 'i', 's', 'h'], ['l', 'a', 'm', 'b'], ['.']], 'tag': ['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']}


In [10]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device, sort=False)

# Construire le modèle

Il y a deux nouvelles couches a ajouter basé sur le modèle de [Ma & Hovy (2016)](https://arxiv.org/abs/1603.01354):

1. embedding de caractères
2. CNN de l'embedding de caractères

In [11]:
class BiLSTM(nn.Module):

    def __init__(self,
                 input_dim,
                 embedding_dim,
                 char_emb_dim,  
                 char_input_dim,  
                 char_cnn_filter_num,  
                 char_cnn_kernel_size, 
                 hidden_dim,
                 output_dim,
                 lstm_layers,
                 emb_dropout,
                 cnn_dropout,  
                 lstm_dropout,
                 fc_dropout,
                 word_pad_idx,
                 char_pad_idx):  
        super().__init__()
        
        # Couche 1A: Word Embedding
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(
            num_embeddings=input_dim,
            embedding_dim=embedding_dim,
            padding_idx=word_pad_idx
        )
        self.emb_dropout = nn.Dropout(emb_dropout) 
        
        # Couche 1B: Char Embedding-CNN
        self.char_emb_dim = char_emb_dim
        self.char_emb = nn.Embedding(
            num_embeddings=char_input_dim,
            embedding_dim=char_emb_dim,
            padding_idx=char_pad_idx
        )
        self.char_cnn = nn.Conv1d(
            in_channels=char_emb_dim,
            out_channels=char_emb_dim * char_cnn_filter_num,
            kernel_size=char_cnn_kernel_size,
            groups=char_emb_dim  # different 1d conv for each embedding dim
        )
        self.cnn_dropout = nn.Dropout(cnn_dropout)
        
        # Couche 2: BiLSTM
        self.lstm = nn.LSTM(
            input_size= embedding_dim + (char_emb_dim * char_cnn_filter_num),
            hidden_size=hidden_dim,
            num_layers=lstm_layers,
            bidirectional=True,
            dropout=lstm_dropout if lstm_layers > 1 else 0
        )
        
        # Couche 3: Fully-connected
        self.fc_dropout = nn.Dropout(fc_dropout)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)  # times 2 for bidirectional
        # init weights from normal distribution
        for name, param in self.named_parameters():
            nn.init.normal_(param.data, mean=0, std=0.1)

    def forward(self, words, chars):
        
        # words = [sentence length, batch size]
        # chars = [batch size, sentence length, word length)
        # embedding_out = [sentence length, batch size, embedding dim]
        embedding_out = self.emb_dropout(self.embedding(words))

        # char_emb_out = [batch size, sentence length, word length, char emb dim]
        char_emb_out = self.emb_dropout(self.char_emb(chars))
        batch_size, sent_len, word_len, char_emb_dim = char_emb_out.shape
        char_cnn_max_out = torch.zeros(batch_size, sent_len, self.char_cnn.out_channels)
        
        # pour l'embedding des caractères, on doit itérer sur les phrases       
        for sent_i in range(sent_len):
            # sent_char_emb = [batch size, word length, char emb dim]
            sent_char_emb = char_emb_out[:, sent_i, :, :]  # get the character field of sent i
            # sent_char_emb_p = [batch size, char emb dim, word length]
            sent_char_emb_p = sent_char_emb.permute(0, 2, 1)  # the channel (char emb dim) has to be the last dimension
            # char_cnn_sent_out = [batch size, out channels * char emb dim, word length - kernel size + 1]
            char_cnn_sent_out = self.char_cnn(sent_char_emb_p)
            char_cnn_max_out[:, sent_i, :], _ = torch.max(char_cnn_sent_out, dim=2)  # max pooling over the word length dimension
        char_cnn = self.cnn_dropout(char_cnn_max_out)

        # char_cnn_p = [sentence length, batch size, char emb dim * num filter]
        char_cnn_p = char_cnn.permute(1, 0, 2).to(device)
        word_features = torch.cat((embedding_out, char_cnn_p), dim=2)

        # lstm_out = [sentence length, batch size, hidden dim * 2]
        lstm_out, _ = self.lstm(word_features)
        # ner_out = [sentence length, batch size, output dim]
        ner_out = self.fc(self.fc_dropout(lstm_out))
        return ner_out

    def init_embeddings(self, char_pad_idx, word_pad_idx, pretrained=None, freeze=True):
        # initialise l'embedding pour les tokens pad à zéro
        self.embedding.weight.data[word_pad_idx] = torch.zeros(self.embedding_dim)
        self.char_emb.weight.data[char_pad_idx] = torch.zeros(self.char_emb_dim)
        if pretrained is not None:
            self.embedding = nn.Embedding.from_pretrained(
                embeddings=torch.as_tensor(pretrained),
                padding_idx=word_pad_idx,
                freeze=freeze
            )

    def count_parameters(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

In [12]:
# prepare padding index to be ignored during model training/evaluation
TEXT_PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
CHAR_PAD_IDX = CHAR.vocab.stoi[CHAR.pad_token]  
TAG_PAD_IDX = TAG.vocab.stoi[TAG.pad_token]

In [13]:
model = BiLSTM(
    input_dim=len(TEXT.vocab),
    embedding_dim=300,
    char_emb_dim=25,
    char_input_dim=len(CHAR.vocab),
    char_cnn_filter_num=5,
    char_cnn_kernel_size=3,
    hidden_dim=64,
    output_dim=len(TAG.vocab),
    lstm_layers=2,
    emb_dropout=0.5,
    cnn_dropout=0.25,
    lstm_dropout=0.1,
    fc_dropout=0.25,
    word_pad_idx=TEXT_PAD_IDX,
    char_pad_idx=CHAR_PAD_IDX
)
model.init_embeddings(
    char_pad_idx=CHAR_PAD_IDX,
    word_pad_idx=TEXT_PAD_IDX ,
    freeze=True
)
print(f"Le modèle a {model.count_parameters():,} paramètres à entraîner.")
print(model)

Le modèle a 3,640,260 paramètres à entraîner.
BiLSTM(
  (embedding): Embedding(10952, 300, padding_idx=1)
  (emb_dropout): Dropout(p=0.5, inplace=False)
  (char_emb): Embedding(86, 25, padding_idx=1)
  (char_cnn): Conv1d(25, 125, kernel_size=(3,), stride=(1,), groups=25)
  (cnn_dropout): Dropout(p=0.25, inplace=False)
  (lstm): LSTM(425, 64, num_layers=2, dropout=0.1, bidirectional=True)
  (fc_dropout): Dropout(p=0.25, inplace=False)
  (fc): Linear(in_features=128, out_features=10, bias=True)
)


# Entraînement


In [14]:
optimizer = optim.Adam(model.parameters())

criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)

model = model.to(device)
criterion = criterion.to(device)


def f1_loss(preds, y, tag_pad_idx):
    '''
    Retourne le score F1
    '''  
     
    max_preds = preds.argmax(dim = 1, keepdim = True) 
    non_pad_elements = (y != tag_pad_idx).nonzero()
    y_pred = (max_preds[non_pad_elements])
    y_true = (y_pred == y[non_pad_elements]).float() 
            
    tp = (y_true * y_pred).sum().float()
    tn = ((1 - y_true) * (1 - y_pred)).sum().float()
    fp = ((1 - y_true) * y_pred).sum().float()
    fn = (y_true * (1 - y_pred)).sum().float()
    
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    
    if (tp + fn) == 0 or (tp + fp) == 0 or (recall + precision == 0):
        f1 = torch.zeros(1)
    else:
        f1 = 2* (precision*recall) / (precision + recall)
    
    return f1

def train(model, iterator, optimizer, criterion, tag_pad_idx):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        text = batch.text
        tags = batch.tag
        chars = batch.char 
        optimizer.zero_grad()
        
        #text = [sent len, batch size]
        
        predictions = model(text,chars)
        
        #predictions = [sent len, batch size, output dim]
        #tags = [sent len, batch size]
        
        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)
        
        #predictions = [sent len * batch size, output dim]
        #tags = [sent len * batch size]
        
        loss = criterion(predictions, tags)
                
        acc = f1_loss(predictions, tags, tag_pad_idx)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion, tag_pad_idx):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text = batch.text
            tags = batch.tag
            chars = batch.char
            predictions = model(text,chars)
            
            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)
            
            loss = criterion(predictions, tags)
            
            acc = f1_loss(predictions, tags, tag_pad_idx)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

N_EPOCHS = 2

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, TAG_PAD_IDX)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, TAG_PAD_IDX)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

W torch.Size([10952, 300])
I torch.Size([41, 128])
W torch.Size([86, 25])
I torch.Size([128, 41, 14])
W torch.Size([10952, 300])
I torch.Size([48, 128])
W torch.Size([86, 25])
I torch.Size([128, 48, 16])
W torch.Size([10952, 300])
I torch.Size([52, 128])
W torch.Size([86, 25])
I torch.Size([128, 52, 27])
W torch.Size([10952, 300])
I torch.Size([49, 128])
W torch.Size([86, 25])
I torch.Size([128, 49, 17])
W torch.Size([10952, 300])
I torch.Size([45, 128])
W torch.Size([86, 25])
I torch.Size([128, 45, 20])
W torch.Size([10952, 300])
I torch.Size([55, 128])
W torch.Size([86, 25])
I torch.Size([128, 55, 15])
W torch.Size([10952, 300])
I torch.Size([41, 128])
W torch.Size([86, 25])
I torch.Size([128, 41, 14])
W torch.Size([10952, 300])
I torch.Size([56, 128])
W torch.Size([86, 25])
I torch.Size([128, 56, 24])
W torch.Size([10952, 300])
I torch.Size([51, 128])
W torch.Size([86, 25])
I torch.Size([128, 51, 15])
W torch.Size([10952, 300])
I torch.Size([43, 128])
W torch.Size([86, 25])
I torch.

W torch.Size([10952, 300])
I torch.Size([44, 128])
W torch.Size([86, 25])
I torch.Size([128, 44, 14])
W torch.Size([10952, 300])
I torch.Size([51, 128])
W torch.Size([86, 25])
I torch.Size([128, 51, 20])
W torch.Size([10952, 300])
I torch.Size([41, 128])
W torch.Size([86, 25])
I torch.Size([128, 41, 18])
W torch.Size([10952, 300])
I torch.Size([48, 128])
W torch.Size([86, 25])
I torch.Size([128, 48, 14])
W torch.Size([10952, 300])
I torch.Size([55, 128])
W torch.Size([86, 25])
I torch.Size([128, 55, 16])
W torch.Size([10952, 300])
I torch.Size([57, 128])
W torch.Size([86, 25])
I torch.Size([128, 57, 20])
W torch.Size([10952, 300])
I torch.Size([41, 128])
W torch.Size([86, 25])
I torch.Size([128, 41, 14])
W torch.Size([10952, 300])
I torch.Size([60, 128])
W torch.Size([86, 25])
I torch.Size([128, 60, 16])
W torch.Size([10952, 300])
I torch.Size([44, 128])
W torch.Size([86, 25])
I torch.Size([128, 44, 21])
W torch.Size([10952, 300])
I torch.Size([45, 128])
W torch.Size([86, 25])
I torch.

W torch.Size([10952, 300])
I torch.Size([48, 128])
W torch.Size([86, 25])
I torch.Size([128, 48, 16])
W torch.Size([10952, 300])
I torch.Size([40, 128])
W torch.Size([86, 25])
I torch.Size([128, 40, 17])
W torch.Size([10952, 300])
I torch.Size([52, 128])
W torch.Size([86, 25])
I torch.Size([128, 52, 14])
W torch.Size([10952, 300])
I torch.Size([45, 128])
W torch.Size([86, 25])
I torch.Size([128, 45, 15])
W torch.Size([10952, 300])
I torch.Size([53, 128])
W torch.Size([86, 25])
I torch.Size([128, 53, 27])
W torch.Size([10952, 300])
I torch.Size([51, 128])
W torch.Size([86, 25])
I torch.Size([128, 51, 15])
W torch.Size([10952, 300])
I torch.Size([52, 128])
W torch.Size([86, 25])
I torch.Size([128, 52, 16])
W torch.Size([10952, 300])
I torch.Size([51, 128])
W torch.Size([86, 25])
I torch.Size([128, 51, 16])
W torch.Size([10952, 300])
I torch.Size([43, 128])
W torch.Size([86, 25])
I torch.Size([128, 43, 14])
W torch.Size([10952, 300])
I torch.Size([60, 128])
W torch.Size([86, 25])
I torch.

W torch.Size([10952, 300])
I torch.Size([48, 128])
W torch.Size([86, 25])
I torch.Size([128, 48, 20])
W torch.Size([10952, 300])
I torch.Size([51, 128])
W torch.Size([86, 25])
I torch.Size([128, 51, 15])
W torch.Size([10952, 300])
I torch.Size([48, 128])
W torch.Size([86, 25])
I torch.Size([128, 48, 61])
W torch.Size([10952, 300])
I torch.Size([48, 128])
W torch.Size([86, 25])
I torch.Size([128, 48, 16])
W torch.Size([10952, 300])
I torch.Size([44, 128])
W torch.Size([86, 25])
I torch.Size([128, 44, 24])
W torch.Size([10952, 300])
I torch.Size([55, 128])
W torch.Size([86, 25])
I torch.Size([128, 55, 17])
W torch.Size([10952, 300])
I torch.Size([44, 128])
W torch.Size([86, 25])
I torch.Size([128, 44, 25])
W torch.Size([10952, 300])
I torch.Size([39, 10])
W torch.Size([86, 25])
I torch.Size([10, 39, 12])
W torch.Size([10952, 300])
I torch.Size([45, 128])
W torch.Size([86, 25])
I torch.Size([128, 45, 24])
W torch.Size([10952, 300])
I torch.Size([48, 128])
W torch.Size([86, 25])
I torch.Si

In [15]:
#model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, TAG_PAD_IDX)

print(f'Test Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}%')

W torch.Size([10952, 300])
I torch.Size([72, 128])
W torch.Size([86, 25])
I torch.Size([128, 72, 14])
W torch.Size([10952, 300])
I torch.Size([124, 128])
W torch.Size([86, 25])
I torch.Size([128, 124, 14])
W torch.Size([10952, 300])
I torch.Size([44, 128])
W torch.Size([86, 25])
I torch.Size([128, 44, 15])
W torch.Size([10952, 300])
I torch.Size([44, 128])
W torch.Size([86, 25])
I torch.Size([128, 44, 14])
W torch.Size([10952, 300])
I torch.Size([38, 128])
W torch.Size([86, 25])
I torch.Size([128, 38, 18])
W torch.Size([10952, 300])
I torch.Size([59, 128])
W torch.Size([86, 25])
I torch.Size([128, 59, 16])
W torch.Size([10952, 300])
I torch.Size([42, 128])
W torch.Size([86, 25])
I torch.Size([128, 42, 17])
W torch.Size([10952, 300])
I torch.Size([55, 128])
W torch.Size([86, 25])
I torch.Size([128, 55, 20])
W torch.Size([10952, 300])
I torch.Size([55, 128])
W torch.Size([86, 25])
I torch.Size([128, 55, 20])
W torch.Size([10952, 300])
I torch.Size([48, 128])
W torch.Size([86, 25])
I torc

On remarque que l'accuracy est n'est pas beaucoup meilleure que précédemment.

In [16]:
def tag_sentence(model, device, sentence, text_field, tag_field, char_field):
    
    model.eval()
    
    if isinstance(sentence, str):
        nlp = spacy.load('en')
        tokens = [token.text for token in nlp(sentence)]
    else:
        tokens = [token for token in sentence]

    if text_field.lower:
        tokens = [t.lower() for t in tokens]
        
    max_word_len = max([len(token) for token in tokens])
    numericalized_chars = []
    char_pad_id = char_field.vocab.stoi[CHAR.pad_token] 
    for token in tokens:
        numericalized_chars.append(
                [char_field.vocab.stoi[char] for char in token]
                + [char_pad_id for _ in range(max_word_len - len(token))]
                )
    numericalized_tokens = [text_field.vocab.stoi[t] for t in tokens]

    unk_idx = text_field.vocab.stoi[text_field.unk_token]
    
    unks = [t for t, n in zip(tokens, numericalized_tokens) if n == unk_idx]
    
    token_tensor = torch.LongTensor(numericalized_tokens)
    
    token_tensor = token_tensor.unsqueeze(-1).to(device)
    char_tensor = torch.as_tensor(numericalized_chars)
    char_tensor = char_tensor.unsqueeze(0).to(device) 
    
    predictions = model(token_tensor, char_tensor)
    
    top_predictions = predictions.argmax(-1)
    
    predicted_tags = [tag_field.vocab.itos[t.item()] for t in top_predictions]
    
    return tokens, predicted_tags, unks

In [17]:
example_index = 1

sentence = vars(train_data.examples[example_index])['text']
actual_tags = vars(train_data.examples[example_index])['tag']

print(sentence)

['eu', 'rejects', 'german', 'call', 'to', 'boycott', 'british', 'lamb', '.']


In [18]:
tokens, pred_tags, unks = tag_sentence(model, 
                                       device, 
                                       sentence, 
                                       TEXT, 
                                       TAG,
                               
                                       CHAR)

print(unks)

W torch.Size([10952, 300])
I torch.Size([9, 1])
W torch.Size([86, 25])
I torch.Size([1, 9, 7])
['rejects']


In [19]:
print("Pred. Tag\tActual Tag\tCorrect?\tToken\n")

for token, pred_tag, actual_tag in zip(tokens, pred_tags, actual_tags):
    correct = '✔' if pred_tag == actual_tag else '✘'
    print(f"{pred_tag}\t\t{actual_tag}\t\t{correct}\t\t{token}")

Pred. Tag	Actual Tag	Correct?	Token

B-ORG		B-ORG		✔		eu
O		O		✔		rejects
B-MISC		B-MISC		✔		german
O		O		✔		call
O		O		✔		to
O		O		✔		boycott
B-MISC		B-MISC		✔		british
O		O		✔		lamb
O		O		✔		.


In [22]:
sentence = 'The will deliver a speech about the conflict in North Korea tomorrow in New York with my friend Mary Kate.'

tokens, tags, unks = tag_sentence(model, 
                                  device, 
                                  sentence, 
                                  TEXT, 
                                  TAG,
                                  CHAR)

print(unks)

W torch.Size([10952, 300])
I torch.Size([21, 1])
W torch.Size([86, 25])
I torch.Size([1, 21, 8])
['kate']


In [23]:
print("Pred. Tag\tToken\n")


for token, tag in zip(tokens, tags):
    print(f"{tag}\t\t{token}")

Pred. Tag	Token

O		the
O		will
O		deliver
O		a
O		speech
O		about
O		the
O		conflict
O		in
B-MISC		north
B-LOC		korea
O		tomorrow
O		in
B-LOC		new
B-LOC		york
O		with
O		my
O		friend
B-PER		mary
I-PER		kate
O		.
